# Map Layer Update: TEMPLATE

This notebook serves as template and guidance for some common updates on map layers: tile cache, decode function, legend and other descriptions...  

Here, these tasks are performed using [LMIpy](https://lmipy.readthedocs.io/en/latest/), the tool developed by **Vizzuality** to interact with [Resource Watch API](http://api.resourcewatch.org/) assets. These actions and changes could also be done by other API interaction methods, such as Python `requests` library or Postman



### Load libraries

In [ ]:
import os
import json
import geopandas as gpd
from IPython.display import display
import getpass
import requests

In [ ]:
# Install LMIpy if needed
#!pip install LMIPy
#!python setup.py develop 

from IPython.display import clear_output
clear_output()

import LMIPy as lmi
from LMIPy import utils, dataAPI

print(f'LMI ver. {lmi.__version__} ready!')

## Authentiction  
To modify Resource Watch API's assets, the user needs a RW API account ([see here on creation](https://resource-watch.github.io/doc-api/quickstart.html#2-create-an-account-with-the-rw-api)) with editor privileges, and use it to retrieve the API token key.

In [ ]:
email = getpass.getpass('Login email:')
password = getpass.getpass('Login password:')

In [ ]:
payload = {
    "email": f"{email}",
    "password": f"{password}"
}

url = f'https://api.resourcewatch.org/auth/login'

headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)

API_TOKEN = r.json().get('data').get('token')

## Get dataset of interest  
Datasets can contain one or more layers, and included higher-level attributes. It is suggested to first retrieve the dataset containing the layer to modify, and then access the layer.  

To find the desired dataset, we search it using LMIpy and defining:  
- a keyword to search  
- the app it belongs to (gfw in this case)  
- the type of asset (dataset, layer or both)  
- the environment (staging, preproduction or production)

**Find staging dataset**

In [ ]:
search_term = 'loss'
app = ['gfw']
env = 'staging'
#object_type = ['layer', 'dataset']
object_type = 'dataset' #search for dataset only
collection = lmi.Collection(search=search_term, app=app, env=env, object_type=object_type)
collection

**Check production datasets for reference**

In [ ]:
#search_term = 'loss'
#app = ['gfw']
env = 'production'
#object_type = ['layer', 'dataset']
object_type = 'dataset'
collection_prod = lmi.Collection(search=search_term, app=app, env=env, object_type=object_type)
collection_prod

In [ ]:
ds_prod = collection_prod[25] #get index from list
#prod_example = lmi.Dataset(id_hash = ) #alternative if we already have the dataset id
ds_prod

## Check dataset properties

In [ ]:
ds =  collection[] #get index from list
ds

In [ ]:
ds.attributes

## Get associated layer id

In [ ]:
ds.layers

In [ ]:
lay = ds.layers[]
lay

## Check properties in production  
Get a first view of the layer's structure in production. Also, check if the layer uses a decode function (only applies to raster data), update if necessary.

In [ ]:
ds_prod.layers

In [ ]:
prod_lay = ds_prod.layers[0]

In [ ]:
prod_lay.attributes['layerConfig']

Take note if necessary  
```"decode_function": " "```

## Update layer info
Some common updates include:  
- Update tile cache URL  
- Update legend info (text, colors, selectors)
- Update / check metadata  

Where to access to the information / parameters to update:  
- `attributes`: Layer name (title in the legend) and status (published: True/False)  
- `layerConfig`: tile cache URL, decoding function, max zoom, time/threshold selectors and parameter used  
- `legendConfig`: legend type and components (colors, categories), selector's sentence, extra text info  
- `interactionConfig`: layer selector parameters (when >1 layer on dataset), analytics tracking keys, metadata endpoint, layer slug id



**Check staging layer properties**

In [ ]:
#Layer config attributes
lay.attributes['layerConfig']

In [ ]:
# Legend config attributes
lay.attributes['legendConfig']

### Example of layer update  
The payload provided must include the whole object (all the parameters) of the category, in this case `layerConfig`. Any parameter / value can be modified as desired inside that (for example, provide a new tile cache URL / version)  
This operation requires the API token passed as `token` parameter.

In [ ]:
lay.update(update_params={'layerConfig':{}
}, token=API_TOKEN)

## Changes at dataset level

### Publish / unpublish dataset

In [ ]:
ds.update(update_params={
    'published': True,
}, token=API_TOKEN)

### Update dataset metadata  
Dataset metadata contains the information being displayed in the contents index (left hand menu in the platform to select layers).  
Some key parameters are:  
- name: Name of the dataset displayed in the contents index  
- description: text in the information icon  
- citation: information displayed under the dataset name  
- color: toggle button color  

In [ ]:
ds.metadata

In [ ]:
metas = ds.metadata[0]
metas.attributes

**To update metadata, include the whole object as payload and modify as needed**  
**NOTE: to check the changes on the metadata, the dataset needs to be reloaded (create `ds` and `metas` variables again**)

In [ ]:
metas.update(update_params={}, token = API_TOKEN)

## Deploy to production  


### 1) load staging and production datasets (if not already) via id  

In [ ]:
ds = lmi.Dataset('')
ds

In [ ]:
ds_prod = lmi.Dataset('')
ds_prod

### 2) Clone staging dataset to production  
Update dataset asset name.  
Change environment to `production` afterwards (it does not work during clonning)

In [ ]:
dataset_name = 'SET NAME (v202205)'


ds_clone = ds.clone(
    token=API_TOKEN,
    env='production',
    dataset_params={
        'name': dataset_name,
        'application': ['gfw']
    },
    clone_children=True
)
ds_clone.update(update_params={'env':'production'}, token=API_TOKEN)

### 3) Unpublish old production dataset  
Update name to indicate old dataset

In [ ]:
ds_prod.update(update_params={
    'name': 'SET NAME (DEPRECATED)',
    'published':False}, token=API_TOKEN)